In [ ]:
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import os

# BLIP 모델 로딩 (이미지 캡션 생성 모델)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model_blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# 이미지 로딩 및 캡션 생성 함수
def generate_caption(image_path):
    # 이미지 경로 확인
    if not os.path.exists(image_path):
        print(f"파일이 존재하지 않습니다: {image_path}")
        return None
    
    try:
        # 이미지 열기
        image = Image.open(image_path)
    except Exception as e:
        print(f"이미지 열기 오류: {e}")
        return None
    
    # 이미지 전처리
    inputs = processor(images=image, return_tensors="pt")
    
    # BLIP 모델로 이미지 캡션 생성
    with torch.no_grad():
        output = model_blip.generate(**inputs)
    
    # 캡션 텍스트 반환
    caption = processor.decode(output[0], skip_special_tokens=True)
    return caption

# 로컬 이미지 경로
image_path = "./data/image_1.png"  # 이미지 경로 (절대 경로로 설정해보세요)

# 이미지 캡션 생성
caption = generate_caption(image_path)

# 캡션이 정상적으로 생성된 경우에만 진행
if caption:
    # GPT-4 프롬프트 생성
    prompt_template = """
    You are an expert in recommending perfumes based on scent descriptions. Analyze the following description, find matching perfumes from Fragrantica's notes database, and provide recommendations. Additionally, give a concise yet detailed reason for each recommendation in Korean, focusing on emotional depth and vivid imagery.

    Please respond directly with the recommendations and explanations in Korean. Do not include introductory or transitional phrases.

    {input_text}

    Do not provide any additional information.
    """
    
    # Langchain을 통해 GPT-4 모델을 사용하여 향수 추천 받기
    try:
        # ChatOpenAI 객체 생성
        model_chat = ChatOpenAI(model="gpt-4o")
        
        # 프롬프트 템플릿을 Langchain PromptTemplate으로 설정
        prompt = PromptTemplate(input_variables=["input_text"], template=prompt_template)

        # 포맷된 텍스트를 생성하여 전달
        formatted_prompt = prompt.format(input_text=caption)

        # GPT-4 모델에 포맷된 프롬프트 텍스트 전달
        response = model_chat.invoke(formatted_prompt)  # 텍스트 형태로 전달
        
        # 향수 추천 출력
        print(f"향수 추천 답변 : {response}")
    
    except Exception as e:
        print(f"GPT-4 요청 중 오류 발생: {e}")
else:
    print("이미지 캡션 생성 실패")

In [13]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ------- -------------------------------- 2.9/16.1 MB 16.7 MB/s eta 0:00:01
   ---------------------------------- ----- 13.9/16.1 MB 37.9 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 30.6 MB/s eta 0:00:00


In [11]:
import requests
from bs4 import BeautifulSoup
import re
import mysql.connector

# 1. 페이지 요청 및 데이터 추출
url = "https://www.bysuco.com/product/show/10155"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 데이터 추출
desc_wrap = soup.find("div", class_="descWrap")
brand = soup.find("a", class_="tit")
title = soup.find("h3", class_="desc ellipsisTwo")

if desc_wrap:
    text_content = desc_wrap.get_text(separator=" ", strip=True)
else:
    print("descWrap 요소를 찾을 수 없습니다.")

brand_name = brand.get_text(separator=" ", strip=True) if brand else "Unknown"
title = title.get_text(separator=" ", strip=True) if title else "Unknown"

# 텍스트에서 각 항목 추출
concentration = re.search(r"\[부향률\]\s*-\s*([^\n]+)", text_content)
concentration = concentration.group(1).strip() if concentration else "Unknown"

top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", text_content)
top_note = top_note.group(1).strip() if top_note else "Unknown"

middle_note = re.search(r"미들 노트:([^\n]+)", text_content)
middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

base_note = re.search(r"베이스 노트:([^\n]+)", text_content)
base_note = base_note.group(1).strip() if base_note else "Unknown"

description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", text_content)
description = description.group(1).strip() if description else "Unknown"

# 2. MySQL 연결
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

try:
    with connection.cursor() as cursor:
        # 테이블 생성
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS top_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                top_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS middle_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                middle_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS base_note (
                id INT AUTO_INCREMENT PRIMARY KEY,
                base_note TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS perfume (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(50),
                brand VARCHAR(50),
                grade VARCHAR(50),
                description TEXT,
                top_note TEXT,
                middle_note TEXT,
                base_note TEXT
            )
        ''')

        # 데이터 삽입
        cursor.execute("INSERT INTO perfume (name, brand, grade, description, top_note, middle_note, base_note) VALUES (%s, %s, %s, %s, %s, %s, %s)", 
                    (title, brand_name, concentration, description, top_note, middle_note, base_note))
        perfume_id = cursor.lastrowid

        # 각 노트 삽입
        if top_note:
            cursor.execute("INSERT INTO top_note (id, top_note) VALUES (%s, %s)", (perfume_id, top_note))
        if middle_note:
            cursor.execute("INSERT INTO middle_note (id, middle_note) VALUES (%s, %s)", (perfume_id, middle_note))
        if base_note:
            cursor.execute("INSERT INTO base_note (id, base_note) VALUES (%s, %s)", (perfume_id, base_note))

        # 변경 사항 저장
        connection.commit()

finally:
    connection.close()

print("데이터가 성공적으로 저장되었습니다!")


데이터가 성공적으로 저장되었습니다!


In [20]:
!pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.bysuco.com/product?num=60&page=1&orderBy=popular&keyword=&kind=bt')
time.sleep(3)

# 상품 목록에서 상세 페이지 링크 추출
product_links = []
elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
for element in elements:
    href = element.get_attribute('href')
    product_links.append(href)

# 각 상품 상세 페이지에서 텍스트 추출
for link in product_links:
    driver.get(link)
    time.sleep(3)

    try:
        # 클래스 textWrap, ellipsisTwo, desc, tit, descWrap 텍스트 추출
        text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
        ellipsis_two = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
        desc = driver.find_element(By.CLASS_NAME, 'desc').text
        tit = driver.find_element(By.CLASS_NAME, 'tit').text
        desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text

        print(f"상품 텍스트: {text_wrap}")
        print(f"ellipsisTwo: {ellipsis_two}")
        print(f"상품 브랜드: {tit}")
        print(f"descWrap: {desc_wrap}")
    except Exception as e:
        print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")

# 드라이버 종료
driver.quit()

상품 텍스트: 
ellipsisTwo: 오드 우드 오 드 퍼퓸
상품 브랜드: 톰 포드
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 우디 / 오우드 / 웜 스파이시

[메인 노트]
- 싱글 노트: 아가우드, 브라질리안 로즈우드, 샌달우드, 카다멈, 바닐라, 사천 페퍼, 베티버, 통카 빈, 앰버

[향 설명]
- 청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
상품 텍스트: 
ellipsisTwo: 화이트 스웨이드 오 드 퍼퓸
상품 브랜드: 톰 포드
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 머스크 / 파우더리 / 레더

[메인 노트]
- 탑 노트: 타임, 티
- 미들 노트: 릴리오브더밸리, 샤프론, 로즈
- 베이스 노트: 스웨이드, 머스크, 샌달우드, 올리바넘, 앰버

[향 설명]
- 가죽의 향이 섞인 머스크가 표현하는 제 2의 피부의 향
상품 텍스트: 
ellipsisTwo: 플레르 드 뽀 오 드 퍼퓸
상품 브랜드: 딥티크
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 머스크 / 파우더리 / 아이리스

[메인 노트]
- 탑 노트: 알데하이드, 핑크 페퍼, 앤젤리카, 베르가못
- 미들 노트: 아이리스, 터키쉬 장미
- 베이스 노트: 머스크, 암브레트, 당근, 앰버그리스, 샌달우드, 레더, 앰버우드

[향 설명]
- 달콤한 숨소리로 로맨틱한 포옹을 하는 듯한 관능적인 체취를 느낄 수 있는 향
상품 텍스트: 
ellipsisTwo: 테싯 오 드 퍼퓸
상품 브랜드: 이솝
descWrap: [부향률] 
- 오 드 퍼퓸

[메인 어코드]
- 시트러스 / 아로마틱 / 프레쉬 스파이시

[메인 노트]
- 탑 노트: 유자, 시트러스
- 미들 노트: 바질
- 베이스 노트: 베티버, 클로브

[향 설명]
- 이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향
상품 텍스트: 
ellipsisTwo: 오르페옹 오 드 퍼퓸
상품 브랜드: 딥티크
descWrap: [부향률] 
-

KeyboardInterrupt: 

In [ ]:
# 향수 디비 저장(image_id 매핑 , 노트 엔티티 따로 저장)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
base_url = 'https://www.bysuco.com/product?num=60&page='
page_number = 1  # 페이지 번호 시작

while True:
    # 각 페이지에 접근
    driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
    time.sleep(1)

    # 상품 목록에서 상세 페이지 링크 추출
    product_links = []
    elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
    
    # 상품이 없으면 루프 종료
    if not elements:
        print("더 이상 상품이 없습니다. 수집을 종료합니다.")
        break

    for element in elements:
        href = element.get_attribute('href')
        product_links.append(href)

    # 각 상품 상세 페이지에서 텍스트 및 이미지 URL 추출
    for link in product_links:
        driver.get(link)
        time.sleep(1)

        try:
            # 페이지가 완전히 로드될 때까지 기다림
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

            # 필요한 데이터 추출
            text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
            name = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
            brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
            desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text
            
            # swiper-wrapper 내 load 클래스의 img src 속성 추출
            swiper_wrapper = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'swiper-wrapper'))
            )
            load_images = swiper_wrapper.find_elements(By.CLASS_NAME, 'load')

            # load 클래스의 img 요소 중 src 속성을 추출
            for img in load_images:
                image_url = img.get_attribute('src')
                print(f"이미지 URL: {image_url}")

            # 텍스트에서 각 항목 추출
            top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
            top_note = top_note.group(1).strip() if top_note else "Unknown"

            middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
            middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

            base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
            base_note = base_note.group(1).strip() if base_note else "Unknown"
            
            single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
            single_note = single_note.group(1).strip() if single_note else "Unknown"

            description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
            description = description.group(1).strip() if description else "Unknown"

            # MySQL 데이터 삽입
            with connection.cursor() as cursor:
                # 테이블 생성
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS image (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    img_url TEXT
                )
                ''')
                
                cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(50),
                    brand VARCHAR(50),
                    description TEXT
                )''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS top_note (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id INT,
                    top_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS middle_note (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id INT,
                    middle_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS base_note (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id INT,
                    base_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                )
                ''')
                
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS single_note (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    perfume_id INT,
                    single_note TEXT,
                    FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                )
                ''')

                try:
                    # 이미지 URL을 image 테이블에 삽입
                    cursor.execute("INSERT INTO image (img_url) VALUES (%s)", (image_url,))
                    
                    # perfume 테이블에 데이터 삽입
                    cursor.execute("""
                        INSERT INTO perfume (name, brand, description) 
                        VALUES (%s, %s, %s)
                    """, (name, brand_name, description))
                    perfume_id = cursor.lastrowid
                    
                    # 각 노트 삽입
                    if top_note != "Unknown":
                        cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
                                    (perfume_id, top_note))
                    if middle_note != "Unknown":
                        cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
                                    (perfume_id, middle_note))
                    if base_note != "Unknown":
                        cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
                                    (perfume_id, base_note))
                    if single_note != "Unknown":
                        cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
                                    (perfume_id, single_note))
                    
                    # 모든 쿼리가 성공적으로 실행되면 커밋
                    connection.commit()

                except Exception as e:
                    # 오류 발생 시 롤백
                    connection.rollback()
                    print(f"데이터 삽입 중 오류 발생: {e}")
                    continue

        except Exception as e:
            print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
            continue  # 에러가 발생하면 다음 상품으로 넘어감

    # 다음 페이지로 이동
    page_number += 1

# 드라이버 종료 및 데이터베이스 연결 종료
driver.quit()
connection.close()

print("모든 데이터가 성공적으로 저장되었습니다!")

이미지 URL: https://cf.bysuco.net/4b722975836a2e17226bfc5af864e36a_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/17016a38206d15968305e8d367dff244.jpg?w=600
이미지 URL: https://cf.bysuco.net/cb645387a50683cb27f5353fbef47592.jpg?w=600
이미지 URL: https://cf.bysuco.net/25aa36916334ba38fde137e024b1727f_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/9d770ad6f633d72b9de1122bb78cec6a.jpg?w=600
이미지 URL: https://cf.bysuco.net/27a38594c0cfdafccde33ae1959f7401.jpg?w=600
이미지 URL: https://cf.bysuco.net/1feebb4d88474e51759d4f8f4e71b603.jpg?w=600
이미지 URL: https://cf.bysuco.net/88b3a4c07b22bb85cd51c27271960828_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/39846c3336d46053b16338ad668185ad.jpg?w=600
이미지 URL: https://cf.bysuco.net/cd442772d69311fbf9dfab6844c141f1.jpg?w=600
이미지 URL: https://cf.bysuco.net/fb4f8e687deadc56c62aac75dd7dccea_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/3442b6acb9a87481b121db01c3760782_cvt.webp?w=600
이미지 URL: https://cf.bysuco.net/73c13c2fd5670d543dc785ca5defac90.jpg?w=600
이미지 URL: http

In [2]:
# 향수 디비 저장(image_id 매핑x , 노트필드 향수엔티티에 추가)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
base_url = 'https://www.bysuco.com/product?num=60&page='
page_number = 1  # 페이지 번호 시작

while True:
    # 각 페이지에 접근
    driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
    time.sleep(1)

    # 상품 목록에서 상세 페이지 링크 추출
    product_links = []
    elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
    
    # 상품이 없으면 루프 종료
    if not elements:
        print("더 이상 상품이 없습니다. 수집을 종료합니다.")
        break

    for element in elements:
        href = element.get_attribute('href')
        product_links.append(href)

    # 각 상품 상세 페이지에서 텍스트 및 이미지 URL 추출
    for link in product_links:
        driver.get(link)
        time.sleep(1)

        try:
            # 페이지가 완전히 로드될 때까지 기다림
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

            # 필요한 데이터 추출
            text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
            name = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
            brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
            desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text
            
            # swiper-wrapper 내 load 클래스의 img src 속성 추출
            # swiper_wrapper = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.CLASS_NAME, 'swiper-wrapper'))
            # )
            # load_images = swiper_wrapper.find_elements(By.CLASS_NAME, 'load')

            # load 클래스의 img 요소 중 src 속성을 추출
            # for img in load_images:
            #     image_url = img.get_attribute('src')
            #     print(f"이미지 URL: {image_url}")

            # 텍스트에서 각 항목 추출
            top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
            top_note = top_note.group(1).strip() if top_note else None

            middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
            middle_note = middle_note.group(1).strip() if middle_note else None

            base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
            base_note = base_note.group(1).strip() if base_note else None
            
            single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
            single_note = single_note.group(1).strip() if single_note else None

            description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
            description = description.group(1).strip() if description else None

            # MySQL 데이터 삽입
            with connection.cursor() as cursor:
                # 테이블 생성
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS image (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    img_url TEXT
                )
                ''')
                
                cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    name VARCHAR(50),
                    brand VARCHAR(50),
                    description TEXT,
                    top_note TEXT,
                    middle_note TEXT,
                    base_note TEXT,
                    single_note TEXT
                )''')

                try:
                    # 이미지 URL을 image 테이블에 삽입
                    # cursor.execute("INSERT INTO image (img_url) VALUES (%s)", (image_url,))
                    
                    # perfume 테이블에 모든 데이터를 한 번에 삽입
                    cursor.execute("""
                        INSERT INTO perfume (
                            name, brand, description, 
                            top_note, middle_note, base_note, single_note
                        ) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                    """, (
                        name, brand_name, description,
                        top_note, middle_note, base_note, single_note
                    ))
                    
                    # 모든 쿼리가 성공적으로 실행되면 커밋
                    connection.commit()

                except Exception as e:
                    # 오류 발생 시 롤백
                    connection.rollback()
                    print(f"데이터 삽입 중 오류 발생: {e}")
                    continue

        except Exception as e:
            print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
            continue  # 에러가 발생하면 다음 상품으로 넘어감

    # 다음 페이지로 이동
    page_number += 1

# 드라이버 종료 및 데이터베이스 연결 종료
driver.quit()
connection.close()

print("모든 데이터가 성공적으로 저장되었습니다!")

상세 페이지에서 정보를 추출할 수 없습니다. 오류: Message: 
Stacktrace:
	GetHandleVerifier [0x005DEBD3+24307]
	(No symbol) [0x00568D74]
	(No symbol) [0x0044C323]
	(No symbol) [0x0048DC86]
	(No symbol) [0x0048DECB]
	(No symbol) [0x004CB9F2]
	(No symbol) [0x004AFED4]
	(No symbol) [0x004C9579]
	(No symbol) [0x004AFC26]
	(No symbol) [0x0048219C]
	(No symbol) [0x0048311D]
	GetHandleVerifier [0x00888D93+2818227]
	GetHandleVerifier [0x008E542E+3196750]
	GetHandleVerifier [0x008DD9D2+3165426]
	GetHandleVerifier [0x0067DA70+675216]
	(No symbol) [0x00571B3D]
	(No symbol) [0x0056EA18]
	(No symbol) [0x0056EBB5]
	(No symbol) [0x00561640]
	BaseThreadInitThunk [0x76AAFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7716809E+286]
	RtlGetAppContainerNamedObjectPath [0x7716806E+238]

상세 페이지에서 정보를 추출할 수 없습니다. 오류: Message: 
Stacktrace:
	GetHandleVerifier [0x005DEBD3+24307]
	(No symbol) [0x00568D74]
	(No symbol) [0x0044C323]
	(No symbol) [0x0048DC86]
	(No symbol) [0x0048DECB]
	(No symbol) [0x004CB9F2]
	(No symbol) [0x004AFED4]
	

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.119)
Stacktrace:
	GetHandleVerifier [0x005DEBD3+24307]
	(No symbol) [0x00568D74]
	(No symbol) [0x0044C323]
	(No symbol) [0x0042E00B]
	(No symbol) [0x004B5F6F]
	(No symbol) [0x004C8EC9]
	(No symbol) [0x004AFC26]
	(No symbol) [0x0048219C]
	(No symbol) [0x0048311D]
	GetHandleVerifier [0x00888D93+2818227]
	GetHandleVerifier [0x008E542E+3196750]
	GetHandleVerifier [0x008DD9D2+3165426]
	GetHandleVerifier [0x0067DA70+675216]
	(No symbol) [0x00571B3D]
	(No symbol) [0x0056EA18]
	(No symbol) [0x0056EBB5]
	(No symbol) [0x00561640]
	BaseThreadInitThunk [0x76AAFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7716809E+286]
	RtlGetAppContainerNamedObjectPath [0x7716806E+238]


In [ ]:
# 향료 db만들기(image_id 매핑x)
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time
from mysql.connector import Error

# DB 연결
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='ohgiraffers',
        password='ohgiraffers', 
        database='test_db',
        charset='utf8mb4',
        use_unicode=True
    )
    
    cursor = connection.cursor(buffered=True)
    
    # 모든 테이블 먼저 생성
    # image 테이블 생성 
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS image (
            id INT AUTO_INCREMENT PRIMARY KEY,
            img_url TEXT NOT NULL
        )
    ''')
    
    # line 테이블 생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS line (
            id INT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(255),
            content VARCHAR(255),
            color VARCHAR(50)
        )
    ''')
    
    # spice 테이블 생성 (line_id 제거)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS spice (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            content TEXT
        )
    ''')

    # 테이블 생성 후 line 데이터 삽입
    cursor.execute('''
        INSERT INTO line (name, content, color) VALUES
            ('Spicy', '자극적이며 섹시한 향', 'FF5757'),
            ('Cypre', '격조있고 성숙한 느낌의 개성있는 향', 'FF7F43'),
            ('Fruity', '달콤하고 귀여운 향', 'FFBD43'),
            ('Citrus', '과일 같은 상큼한 향', 'FFE043'),
            ('Green', '상쾌하고 신선한 향', '62D66A'),
            ('Aldehyde', '인공 비누향, 모던하고 우아하면 지적인 향', '98D1FF'),
            ('Aquatic', '들과 바다 시원함이 가득한 향', '56D2FF'),
            ('Fougere', '부드럽고 감미로우며 편안한 향', 'FFD9A6'),
            ('Gourmand', '달달하고 유혹적인 미각의 즐거움을 느낄 수 있는 향', 'A1522C'),
            ('Woody', '고상하고 증후한 느낌의 따뜻하고 부드러운 향', '86390F'),
            ('Oriental', '무거우면서 부드럽고 고혹적인 향, 증후하고 달콤하며 센슈얼한 향', 'C061FF'),
            ('Floral', '화려하고 여성스러운 향', 'FF80C1'),
            ('Musk', '이성에게 어필 가능한 아름다운 향', 'F8E4FF'),
            ('Powdery', '포근한 향', 'FFFFFF'),
            ('Amber', '달콤하고 부드러운 향', 'FFA75A'),
            ('Tobacco Leather', '마초적인 남성의 개성이 강한 매력적인 향', '000000'),
            ('not', 'not', 'not')
    ''')
    
    connection.commit()  # line 데이터 삽입 후 커밋

    url = "https://www.fragrantica.com/notes/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    notes = soup.find_all('div', class_='cell small-6 medium-4 large-3 text-center notebox')

    if not notes:
        print("향료 데이터를 찾지 못했습니다.")
    else:
        for note in notes:
            try:
                a_tag = note.find('a')
                if not a_tag:
                    continue
                    
                name = a_tag.text.strip()
                note_url = a_tag.get('href')
                img_tag = note.find('img')
                image_url = img_tag.get('src') if img_tag else None

                if not image_url:
                    continue

                # 이미지 데이터 삽입
                cursor.execute("INSERT INTO image (img_url) VALUES (%s)", (image_url,))

                additional_info = "No additional info"
                if note_url:
                    full_note_url = f"https://www.fragrantica.com{note_url}" if not note_url.startswith('http') else note_url
                    note_response = requests.get(full_note_url, headers=headers)
                    if note_response.status_code == 200:
                        note_soup = BeautifulSoup(note_response.text, 'html.parser')
                        callout_div = note_soup.find('div', class_='cell callout')
                        if callout_div:
                            additional_info = callout_div.text.strip()
                            # "Odor profile:" 제거
                            additional_info = additional_info.replace('Odor profile:', '').strip()
                    time.sleep(1)

                # spice 데이터 삽입 (line_id 제외)
                cursor.execute(
                    "INSERT INTO spice (name, content) VALUES (%s, %s)",
                    (name, additional_info)
                )
                print(f"데이터 삽입 성공 - Name: {name}")
                
                connection.commit()  # 각 항목마다 커밋
                
            except Exception as e:
                print(f"항목 처리 중 오류 발생: {e}")
                connection.rollback()
                continue

        print("모든 데이터가 성공적으로 저장되었습니다!")

except Exception as e:
    print(f"프로그램 실행 중 오류 발생: {e}")
    if connection:
        connection.rollback()

finally:
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")

데이터 삽입 성공 - Name: Bergamot
데이터 삽입 성공 - Name: Bigarade
데이터 삽입 성공 - Name: Bitter Orange
데이터 삽입 성공 - Name: Blood Orange
데이터 삽입 성공 - Name: Buddha's hand
데이터 삽입 성공 - Name: Calamansi
데이터 삽입 성공 - Name: Candied Lemon
데이터 삽입 성공 - Name: Chen Pi
데이터 삽입 성공 - Name: Chinotto
데이터 삽입 성공 - Name: Citron
데이터 삽입 성공 - Name: Citrus Water
데이터 삽입 성공 - Name: Citruses
데이터 삽입 성공 - Name: Clementine
데이터 삽입 성공 - Name: Finger Lime
데이터 삽입 성공 - Name: Grapefruit
데이터 삽입 성공 - Name: Grapefruit Leaf
데이터 삽입 성공 - Name: Grapefruit Peel
데이터 삽입 성공 - Name: Green Tangerine
데이터 삽입 성공 - Name: Hassaku
데이터 삽입 성공 - Name: Hatkora Lemon
데이터 삽입 성공 - Name: Kaffir Lime
데이터 삽입 성공 - Name: Kumquat
데이터 삽입 성공 - Name: Lemon
데이터 삽입 성공 - Name: Lemon Balm
데이터 삽입 성공 - Name: Lemon Myrtle
데이터 삽입 성공 - Name: Lemon Tree
데이터 삽입 성공 - Name: Lemon Verbena
데이터 삽입 성공 - Name: Lemon Zest
데이터 삽입 성공 - Name: Lemongrass
데이터 삽입 성공 - Name: Lime
데이터 삽입 성공 - Name: Limetta
데이터 삽입 성공 - Name: Litsea Cubeba
데이터 삽입 성공 - Name: Mandarin Orange
데이터 삽입 성공 - Name: Mandora
데이터 삽입 

In [28]:
with connection.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS single_note;")
    connection.commit()
    print("spice 테이블 삭제 완료")

# # `image` 테이블 삭제
# with connection.cursor() as cursor:
#     cursor.execute("DROP TABLE IF EXISTS image;")
#     connection.commit()
#     print("image 테이블 삭제 완료")

spice 테이블 삭제 완료


In [15]:
# json 딕셔너리로 변환
import json

with open('C:/Users/20118/Desktop/hyeyeon/line.json', 'r', encoding='utf-8') as file:
    dict_data = json.load(file)

print(type(dict_data))

<class 'list'>


In [22]:
json_string = '{"id" : 1, "name" : "Spicy", "content" : "자극적이며 섹시한 향", "color" : "FF5757"}'
data = json.loads(json_string)
print(data)

{'id': 1, 'name': 'Spicy', 'content': '자극적이며 섹시한 향', 'color': 'FF5757'}
